In [1]:
sc.hadoopConfiguration.set("fs.s3n.awsAccessKeyId",sys.env("AWS_ACCESS_KEY"))
sc.hadoopConfiguration.set("fs.s3n.awsSecretAccessKey", sys.env("AWS_SECRET_KEY"))

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.2.10:4040
SparkContext available as 'sc' (version = 3.0.0, master = local[*], app id = local-1600475097359)
SparkSession available as 'spark'


In [2]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

val schema = StructType(
  Array(
    StructField("ip", StringType, true),
    StructField("app", StringType, true),
    StructField("device", StringType, true),
    StructField("os", StringType, true),
    StructField("channel", StringType, true),
    StructField("click_time", StringType, true),
    StructField("attributed_time", StringType, true),
    StructField("is_attributed", StringType, true) 
  )
)

import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
schema: org.apache.spark.sql.types.StructType = StructType(StructField(ip,StringType,true), StructField(app,StringType,true), StructField(device,StringType,true), StructField(os,StringType,true), StructField(channel,StringType,true), StructField(click_time,StringType,true), StructField(attributed_time,StringType,true), StructField(is_attributed,StringType,true))


In [3]:
// data source: https://www.kaggle.com/matleonard/feature-engineering-data
spark.read.format("csv")
    .schema(schema)
    .load("s3n://renan-test/adtracking/*.csv")
    .limit(10).show()

+------+---+------+---+-------+-------------------+-------------------+-------------+
|    ip|app|device| os|channel|         click_time|    attributed_time|is_attributed|
+------+---+------+---+-------+-------------------+-------------------+-------------+
| 59882| 12|     1| 19|    424|2017-11-06 23:41:12|               null|            0|
|212465| 20|     1| 13|    259|2017-11-06 23:41:13|               null|            0|
| 55771|  1|     1| 19|    377|2017-11-06 23:41:13|               null|            0|
| 27646|  3|     1| 13|    409|2017-11-06 23:41:13|               null|            0|
|192030|  3|     1| 19|    442|2017-11-06 23:41:13|               null|            0|
| 44552|  2|     1| 15|    401|2017-11-06 23:41:13|               null|            0|
| 86607| 10|     1| 19|    113|2017-11-06 23:41:13|               null|            0|
|140917|  9|     1| 19|    215|2017-11-06 23:41:13|2017-11-07 14:50:00|            1|
|204577| 84|     0|607|    101|2017-11-06 23:41:14|201

In [4]:
// val trackingStream =  spark.readStream.format("csv")
//     .schema(schema)
//     .load("s3n://renan-test/adtracking/*.csv")
//     .withColumn("value",to_json(struct($"ip",$"app",$"device",$"os",$"channel",$"click_time",$"attributed_time",$"is_attributed")))
//     .withColumn("key",$"app")
//     .select("key","value")

// trackingStream.writeStream
//     .format("kafka")
//     .option("kafka.bootstrap.servers", "localhost:9092")
//     .option("topic", "tracking")
//     .option("checkpointLocation", "/home/rhuanca/demo/_checkpoints/tracking")
//     .start()
//     .awaitTermination()